# Prediction with ANN

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)



s:\Krish_Naik_GenAI\Udemy_labs\ANN_End to End project\venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# Load the ANN trained model, scaler pickle file, and one-hot encoder pickle file
model = load_model('model_ann_bank_churn.h5')

# load the encoder and scaler

with open('one_hot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example input data for customer churn / banking prediction model

input_data = {
    'CreditScore': 600,        # Customer's credit score
    'Geography': 'France',     # Country where the customer is located
    'Gender': 'Male',          # Customer gender
    'Age': 40,                 # Customer age in years
    'Tenure': 3,               # Number of years the customer has been with the bank
    'Balance': 60000,          # Account balance
    'NumOfProducts': 2,        # Number of bank products the customer is using
    'HasCrCard': 1,            # Has credit card (1 = Yes, 0 = No)
    'IsActiveMember': 1,       # Active customer status (1 = Active, 0 = Inactive)
    'EstimatedSalary': 50000   # Estimated yearly salary
}


In [4]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [5]:
# One hot encode 'Geography' column
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

s:\Krish_Naik_GenAI\Udemy_labs\ANN_End to End project\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0

In [6]:
input_df = pd.DataFrame([input_data])
input_df


CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000

In [7]:
# Encode categorical ' variables
input_df['Gender'] = label_encoder_gender.transform([input_df['Gender']])


s:\Krish_Naik_GenAI\Udemy_labs\ANN_End to End project\venv\Lib\site-packages\sklearn\preprocessing\_label.py:139: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [8]:
input_df

CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France       1   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000

In [9]:
## Concatenate the one-hot encoded columns with the original dataframe and drop the original 'Geography' column
if 'Geography' in input_df.columns:
    input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
else:
    input_df = pd.concat([input_df, geo_encoded_df], axis=1)


In [10]:
input_df

CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600       1   40       3    60000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0

In [11]:
## Scalling the input data
input_data_scaled = scaler.transform(input_df)
input_data_scaled


array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# Predict churn

prediction = model.predict(input_data_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


array([[0.02249002]], dtype=float32)

In [17]:
prediction_proba= prediction[0][0]

In [18]:
prediction_proba

np.float32(0.02249002)

In [20]:
if prediction_proba > 0.5 :
    print("The customer is likely to Churn")
else:
    print("The customer is not likely to Churn")

The customer is not likely to Churn
